In [1]:
#!pip install tensorflow-gpu

In [2]:
import tensorflow as tf
print(tf.__version__)

2.8.0


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [4]:
# Importing the dataset from kaggle
# Installing Kaggle API
!pip install -q kaggle

In [5]:
# create a directory as kaggle
! mkdir -p ~/.kaggle

In [6]:
#copy API key to kaggle directory
! cp kaggle.json ~/.kaggle/

In [7]:
#disable API key
! chmod 600 /root/.kaggle/kaggle.json

In [8]:
# list of kaggle datasets
! kaggle datasets list

ref                                                                   title                                             size  lastUpdated          downloadCount  voteCount  usabilityRating  
--------------------------------------------------------------------  -----------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
muratkokludataset/date-fruit-datasets                                 Date Fruit Datasets                              408KB  2022-04-03 09:25:39          10851       1464  0.9375           
victorsoeiro/netflix-tv-shows-and-movies                              Netflix TV Shows and Movies                        2MB  2022-05-15 00:01:23           2924        106  1.0              
mdmahmudulhasansuzan/students-adaptability-level-in-online-education  Students Adaptability Level in Online Education    6KB  2022-04-16 04:46:28           7372        195  1.0              
paradisejoy/top-hits-spotify-from-20002019   

In [9]:
# importing the dataset
!kaggle datasets download -d tongpython/cat-and-dog

 99% 216M/218M [00:01<00:00, 219MB/s]
100% 218M/218M [00:01<00:00, 194MB/s]


In [10]:
# unzipping the dataset
! unzip -q /content/cat-and-dog.zip

In [11]:
training_dir = "/content/training_set/training_set"
test_dir = "/content/test_set/test_set"

In [12]:
# reshaping the images
img_shape = (128,128,3)

## LOADING THE PRE-TRAINED MODEL (MOBILENETV2)

In [14]:
base_model = tf.keras.applications.MobileNetV2(input_shape=img_shape,include_top=False,weights="imagenet")

9420800/9406464 [==============================] - 0s 0us/step


In [15]:
base_model.summary()

Model: "mobilenetv2_1.00_128"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 128, 128, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 64, 64, 32)   864         ['input_1[0][0]']                
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 64, 64, 32)   128         ['Conv1[0][0]']                  
                                                                                                  
 Conv1_relu (ReLU)              (None, 64, 64, 32)   0           ['bn_Conv1[0][

In [16]:
# freezing the model
base_model.trainable = False

## DEFINING THE CUSTOM HEAD FOR NETWORK

In [17]:
base_model.output

<KerasTensor: shape=(None, 4, 4, 1280) dtype=float32 (created by layer 'out_relu')>

In [18]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D() (base_model.output)

In [19]:
global_average_layer

<KerasTensor: shape=(None, 1280) dtype=float32 (created by layer 'global_average_pooling2d')>

In [20]:
# output layer / prediction layer
prediction_layer = tf.keras.layers.Dense(units=1,activation="sigmoid") (global_average_layer)

## DEFINE THE TRANSFER LEARNING MODEL

In [22]:
model = tf.keras.models.Model(inputs=base_model.input,outputs=prediction_layer)

In [23]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 128, 128, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 64, 64, 32)   864         ['input_1[0][0]']                
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 64, 64, 32)   128         ['Conv1[0][0]']                  
                                                                                                  
 Conv1_relu (ReLU)              (None, 64, 64, 32)   0           ['bn_Conv1[0][0]']         

In [24]:
# compile the model
opt = tf.keras.optimizers.RMSprop(learning_rate = 0.0001)

In [25]:
model.compile(optimizer=opt,loss="binary_crossentropy",metrics=["accuracy"])

## CREATE DATA GENERATORS

In [26]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [33]:
data_gen_train = ImageDataGenerator(rescale=1/255.0)
data_gen_test = ImageDataGenerator(rescale=1/255.0)

In [34]:
train_generator = data_gen_train.flow_from_directory(directory=training_dir,
                                                     target_size=(128,128),
                                                     batch_size=128,
                                                     class_mode="binary")

Found 8005 images belonging to 2 classes.


In [35]:
test_generator = data_gen_test.flow_from_directory(directory=test_dir,
                                                     target_size=(128,128),
                                                     batch_size=128,
                                                     class_mode="binary")

Found 2023 images belonging to 2 classes.


## TRAINING THE MODEL

In [36]:
model.fit_generator(generator=train_generator,epochs=5,validation_data=test_generator)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  """Entry point for launching an IPython kernel.


Epoch 1/5
63/63 [==============================] - 44s 490ms/step - loss: 0.5432 - accuracy: 0.7373 - val_loss: 0.4069 - val_accuracy: 0.8453
Epoch 2/5
63/63 [==============================] - 30s 478ms/step - loss: 0.3333 - accuracy: 0.8770 - val_loss: 0.2742 - val_accuracy: 0.9056
Epoch 3/5
63/63 [==============================] - 30s 474ms/step - loss: 0.2372 - accuracy: 0.9158 - val_loss: 0.2046 - val_accuracy: 0.9357
Epoch 4/5
63/63 [==============================] - 30s 470ms/step - loss: 0.1868 - accuracy: 0.9354 - val_loss: 0.1685 - val_accuracy: 0.9471
Epoch 5/5
63/63 [==============================] - 30s 471ms/step - loss: 0.1582 - accuracy: 0.9447 - val_loss: 0.1462 - val_accuracy: 0.9525


## FINE-TUNING

In [37]:
base_model.trainable=True

In [38]:
len(base_model.layers)

154

In [39]:
fine_tune_at = 100

In [40]:
# freeze the layers before 100
for layer in base_model.layers[:fine_tune_at]:
  layer.trainable=False

In [41]:
# compile the model again
model.compile(optimizer=opt,loss="binary_crossentropy",metrics=["accuracy"])

In [42]:
# train the model
model.fit_generator(generator=train_generator,epochs=5,validation_data=test_generator)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  


Epoch 1/5
63/63 [==============================] - 38s 506ms/step - loss: 0.1226 - accuracy: 0.9498 - val_loss: 0.1217 - val_accuracy: 0.9639
Epoch 2/5
63/63 [==============================] - 30s 478ms/step - loss: 0.0240 - accuracy: 0.9926 - val_loss: 0.2016 - val_accuracy: 0.9570
Epoch 3/5
63/63 [==============================] - 30s 478ms/step - loss: 0.0069 - accuracy: 0.9985 - val_loss: 0.1751 - val_accuracy: 0.9614
Epoch 4/5
63/63 [==============================] - 31s 489ms/step - loss: 0.0040 - accuracy: 0.9988 - val_loss: 0.4920 - val_accuracy: 0.9348
Epoch 5/5
63/63 [==============================] - 30s 477ms/step - loss: 0.0014 - accuracy: 0.9996 - val_loss: 0.3192 - val_accuracy: 0.9565
